<a href="https://colab.research.google.com/github/avhadrohini163/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-26 15:39:29--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.07MB/s    in 0.2s    

2022-02-26 15:39:30 (6.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
|R1PR37BR7G3M6A|          1|            2|          3|   N|                Y|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|                Y|
| R8T6MO75ND212|          5|            0|          0|   N|                Y|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|                N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|                Y|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|                Y|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|     

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total votes equal to or greater than 20.
cleaned_vine_df = vine_df.filter(col("total_votes") >= 20)
cleaned_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|
| RLV4XQ49EBYYU|          1|           56|         65|   N|                Y|
| RPBPAJKFD0T26|          5|           63|         63|   N|                Y|
|R1IHO8C56KBN6C|          1|           61|         64|   N|     

In [18]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
import pyspark.sql.functions as F
cleaned_total_df = cleaned_vine_df.withColumn('result_column',((F.col("helpful_votes") / F.col("total_votes"))*100)).filter('result_column>50')
cleaned_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|            100.0|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|69.18238993710692|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|            100.0|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|            100.0|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|            100.0|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|            100.0|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|82.97872340425532|
| RLV4XQ49EBYYU|          1|           5

In [19]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_paid_df = cleaned_total_df.filter(col("vine") == "Y")
vine_paid_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R3JIGR122X50ZV|          5|         2280|       2314|   Y|                N|98.53068280034573|
|R3KV8P8WLD2KKC|          5|          263|        286|   Y|                N|91.95804195804196|
|R2LFY2ZPJ3J7RH|          4|           36|         38|   Y|                N|94.73684210526315|
|R3P01OXMA0U0CA|          5|          114|        117|   Y|                N|97.43589743589743|
|R2SCMXBD9OQP8Z|          2|           22|         26|   Y|                N|84.61538461538461|
|R2FGS9ECOK5IQN|          5|           22|         23|   Y|                N|95.65217391304348|
| R1V9CO41XS6QY|          2|           56|         58|   Y|                N|96.55172413793103|
|R1LEP1JGXRY2YT|          2|           1

In [21]:
# retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_unpaid_df = cleaned_total_df.filter(col("vine") == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|    result_column|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R243UXMMSSZVS6|          2|          254|        254|   N|                Y|            100.0|
|R2P92EHR0S5GBB|          1|          110|        159|   N|                Y|69.18238993710692|
| RN93Z4XPNAANY|          5|           31|         31|   N|                Y|            100.0|
|R1O74T9XO3OPQ4|          5|           38|         38|   N|                Y|            100.0|
|R3DMIHE5BMUT0R|          3|           21|         21|   N|                Y|            100.0|
|R1QWY17NLHQSB5|          5|           36|         36|   N|                Y|            100.0|
|R3LANI678T2YQS|          5|           39|         47|   N|                N|82.97872340425532|
| RLV4XQ49EBYYU|          1|           5

In [27]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid)
total_review_paid = vine_paid_df.select("vine").count()
total_review_paid

969

In [26]:
five_star_paid = vine_paid_df.filter("star_rating == 5").count()
five_star_paid

430

In [28]:
percent_five_star_paid = (five_star_paid / total_review_paid) * 100
percent_five_star_paid


44.375644994840044

In [29]:
total_review_unpaid = vine_unpaid_df.select("vine").count()
total_review_unpaid

43608

In [30]:
five_star_unpaid = vine_unpaid_df.filter("star_rating == 5").count()
five_star_unpaid

19211

In [31]:
percent_five_star_unpaid = (five_star_unpaid / total_review_unpaid) * 100
percent_five_star_unpaid

44.05384333149881